In [2]:
from models import CQTNet

In [3]:
model = CQTNet()
model.load('checkpoints/CQTNet.pth')
model.to('cuda')


CQTNet(
  (features): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool3)

In [4]:
import torch

In [5]:
from cqt_loader import CQT
data=CQT('test')
dataloaders = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False, num_workers=0)

e:\cs682\CQTNet-master\cqt_loader.py:115: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.mode is 'train':


In [6]:
fingerprints = []
model.eval()
k=0
for (i, label) in dataloaders:
    fingerprint = model(i.to('cuda'))
    fingerprints.append(fingerprint[1])
    break

../data/fma_small_wav/062531.wav
torch.Size([1, 84, 200])


In [15]:
fingerprints = torch.cat(fingerprints, dim=0)

In [16]:
fingerprints = fingerprints.cpu().detach().numpy()

In [ ]:
import numpy as np
np.savez('fingerprints.npz', fingerprints=fingerprints)

In [13]:
a_norm = fingerprints / fingerprints.norm(dim=1)[:, None]
b_norm = a_norm
res = torch.mm(a_norm, b_norm.transpose(0,1))
print(res)
#  0.9978 -0.9986 -0.9985
# -0.8629  0.9172  0.9172

# # -------
# # Let's verify with numpy/scipy if our computations are correct:
# a_n = a.numpy()
# b_n = b.numpy()
# res_n = np.zeros((2, 3))
# for i in range(2):
#     for j in range(3):
#         # cos_sim(u, v) = 1 - cos_dist(u, v)
#         res_n[i, j] = 1 - spatial.distance.cosine(a_n[i], b_n[j])
# print(res_n)
# # [[ 0.9978022  -0.99855876 -0.99854881]
# #  [-0.86285472  0.91716063  0.9172349 ]]

tensor([[1.0000, 0.2983, 0.3166, 0.2973, 0.2820, 0.2616, 0.3345, 0.3291, 0.3459,
         0.1578],
        [0.2983, 1.0000, 0.2310, 0.1897, 0.4196, 0.2790, 0.3648, 0.2355, 0.4599,
         0.2745],
        [0.3166, 0.2310, 1.0000, 0.0841, 0.5765, 0.4697, 0.5737, 0.3272, 0.5295,
         0.3315],
        [0.2973, 0.1897, 0.0841, 1.0000, 0.2014, 0.2915, 0.0105, 0.2281, 0.0479,
         0.1466],
        [0.2820, 0.4196, 0.5765, 0.2014, 1.0000, 0.3836, 0.5430, 0.3151, 0.5215,
         0.2462],
        [0.2616, 0.2790, 0.4697, 0.2915, 0.3836, 1.0000, 0.4352, 0.3422, 0.4314,
         0.3551],
        [0.3345, 0.3648, 0.5737, 0.0105, 0.5430, 0.4352, 1.0000, 0.2994, 0.7874,
         0.3464],
        [0.3291, 0.2355, 0.3272, 0.2281, 0.3151, 0.3422, 0.2994, 1.0000, 0.4434,
         0.5128],
        [0.3459, 0.4599, 0.5295, 0.0479, 0.5215, 0.4314, 0.7874, 0.4434, 1.0000,
         0.3976],
        [0.1578, 0.2745, 0.3315, 0.1466, 0.2462, 0.3551, 0.3464, 0.5128, 0.3976,
         1.0000]], device='c